### Add scraped data into dataframe

In [1]:
import sys
sys.path.append('..')
import seaborn as sns
import numpy as np
import pandas as pd
from db import db, query, query_list

In [2]:
import re

In [3]:
beers = query('SELECT * FROM  beers')

In [4]:
bars = query('SELECT * FROM  bars')

## Clean the Bars Table

In [5]:
bars.sample(5)

,establishment_type,neighborhood,city,bar_name,bar_url
313,Beer Store,None,"Long Branch, NJ",Bell Liquors,https://www.beermenus.com/places/17379-bell-li...
243,Bar,None,None,The Seneca,https://www.beermenus.com/places/30816-the-seneca
110,Beer Store,None,None,NK Organic Foods,https://www.beermenus.com/places/53598-nk-orga...
550,Restaurant,Long Island City,"Queens, NY",M.Wells Steakhouse,https://www.beermenus.com/places/38007-m-wells...
40,Beer Store,None,"Manhattan, NY",Alphabet City Beer Co.,https://www.beermenus.com/places/4860-alphabet...


In [6]:
bars.establishment_type.value_counts(dropna = False)
# All bars should become: Bar, Restaurant, Brewery/Brewpub (one category), Beer Store (will be dropped), or missing.
# The miscellaneous values are due to bad parsing of bars not in NYC so can be dropped

Bar              218
Restaurant       130
Beer Store       127
NaN               50
Brewery           26
Brewpub           12
Restaurant in     11
Bar in             8
Brewery in         2
Brewpub in         2
in Pine            1
in Basking         1
in Flanders        1
in Wantagh         1
Name: establishment_type, dtype: int64

In [7]:
fix_dict = { 
      'Bar'           : 'Bar',
      'Restaurant'    : 'Restaurant',
      'Beer Store'    : 'Drop',
      'Brewery'       : 'Brewery',
      'Brewpub'       : 'Brewery',
      'Restaurant in' : 'Restaurant',
      'Bar in'        : 'Bar',
      'Brewery in'    : 'Brewery',
      'Brewpub in'    : 'Brewery',
      'in Pine'       : 'Drop',
      'in Wantagh'    : 'Drop',
      'in Flanders'   : 'Drop',
      'in Basking'    : 'Drop'
}
    
    
def fix_bar_type(bar_type):
    if np.isnan(bar_type):
        return bar_type
    return fix_dict[bar_type]

In [8]:
bars['establishment_type'] = bars.establishment_type.map(fix_dict)

In [9]:
bars = bars[bars.establishment_type != 'Drop']

In [10]:
# TO DO: Bar Locations

## Clean the Beeers Table

In [11]:
beers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56340 entries, 0 to 56339
Data columns (total 11 columns):
name         56340 non-null object
beer_url     55947 non-null object
beer_type    55586 non-null object
abv          55586 non-null object
size         54555 non-null object
kind         55780 non-null object
price        54101 non-null object
bar_url      56340 non-null object
address      56340 non-null object
grouping     56340 non-null object
origin       53979 non-null object
dtypes: object(11)
memory usage: 4.7+ MB


## Origin

In [12]:
beers.loc[(beers.grouping == 'HOUSE BEERS') & (beers.origin.isna()), 'origin'] = 'New York, NY'

### ABV

In [13]:
abv_origin = beers[['abv', 'origin']]

In [14]:
def fix_abv_origin(row):
    try:
        if not pd.isnull(row.abv):
            abv = float(row.abv.strip('% '))
        else:
            abv = row.abv
    except:
        try:
            assert pd.isnull(row.origin)
        except:
            # print(row.abv, '/', row.origin)
            abv = np.nan
            origin = row.origin
        else:
            origin = row.abv
            abv    = np.nan
    else:
        origin = row.origin
    
    if isinstance(origin, float):
        breakpoint()
        
    if pd.isnull(origin):
        origin = np.nan
    else:
        origin = origin.strip()
        cal_re = '\d+ [cC]al'
        if re.search(cal_re, origin):
            # print(origin)
            origin = np.nan
        
                        
    return pd.Series({'abv' : abv, 'origin' : origin})
                    

In [15]:
abv_origin = abv_origin.apply(fix_abv_origin, axis = 'columns')

In [16]:
beers['abv'] = abv_origin.abv
beers['origin'] = abv_origin.origin

### Size and Kind

 - Throw out any beers measured in metric units
 - Throw out any *bars* where there are bulk sales (Keg, 6 pack)
 - Throw out any "beers" that are too large (Growler, Crowler, etc.)
 - Clean up the kind
 - Throw out any beers that aren't beers (wine, whisky, etc.)

#### Filter odd volume units (kegs, mL, etc.)

In [17]:
size_kinds = beers[['size', 'kind']]

In [18]:
def fix_size_kind(row):
    size = row['size']
    kind = row.kind
    drop_row = False
    drop_bar = False

    if not pd.isnull(size):
        if 'cl' in size or 'ml' in size or 'L' in size:
            drop_row = True
            size = np.nan
        if not pd.isnull(size) and 'Keg' in size:
            drop_bar = True
            size = np.nan
        if not pd.isnull(size) and size.strip() == 'Pint':
            size = 16
        elif not pd.isnull(size):
            assert 'oz' in size
            size = float(size.strip().rstrip('oz'))
            if size > 31:
                drop_row = True

    if not pd.isnull(kind):
        if 'Keg' in kind or 'Pack' in kind or 'Cask' in kind:
            kind = np.nan
            drop_bar = True
        elif 'rowler' in kind or 'Cask' in kind or 'Bottles' in kind or 'Pitcher' in kind:
            drop_row = True
        else:
            kind = kind.strip()
            if kind in ['By the glass', 'Glass', 'Pour']:
                kind = 'Draft'
    
    return pd.Series({'size' : size,
                      'kind' : kind,
                      'drop_row' : drop_row,
                      'drop_bar' : drop_bar})

In [19]:
size_kinds = size_kinds.apply(fix_size_kind, axis = 'columns')

In [20]:
# Dropped the tagged rows and bars

temp = beers.drop(labels = ['size', 'kind'], axis = 'columns')
temp = pd.concat([temp, size_kinds], axis = 'columns')
temp['drop_bar'] = temp.groupby('bar_url')['drop_bar'].apply(
                        lambda s : pd.Series(data = s.any(), index = s.index)
                   )

beers = temp[~temp.drop_bar & ~temp.drop_row]

In [21]:
# Drop rows that aren't beers

is_beer = beers.beer_url.str.contains('/beers/').fillna(False)
beers = beers[is_beer]

In [22]:
beers = beers[beers.grouping != 'SOFT DRINKS']

In [23]:
# Drop columns that are no longer necessary

beers.drop(['drop_row', 'drop_bar', 'grouping'], axis = 'columns', inplace = True)

In [24]:
beers.loc[:,'price'] = beers.price.astype(float, errors = 'ignore')

In [25]:
# Fix one beer that was written as $700 for $7.00
beers.loc[beers['price'] == 700, 'price'] = 7

## Bring in the bars data

In [26]:
df = beers.merge(bars, how = 'inner', on = 'bar_url', validate = 'm:1')

## Find the county and keep only New York

In [27]:
# Fix one bad address
bad_address = df.loc[df.address.str.slice(start = -5) == 'kebox', 'address'].iloc[0]
bad_address = bad_address[:bad_address.find('Bucket')]
df.loc[df.address.str.slice(start = -5) == 'kebox', 'address'] = bad_address

In [28]:
# Get the zip code and the zip code table

df['zip_code'] = df.address.str.slice(start = -5).astype(int)

zip_code_table = pd.read_csv('ny_zip_codes.csv').rename(lambda s : s.lower().replace(' ', '_'), axis = 'columns')

zip_code_table = zip_code_table[['county_name', 'zip_code']]

In [29]:
df = df.merge(zip_code_table, on = 'zip_code', how = 'left')

In [30]:
df = df[df.county_name.isin(['New York', 'Kings', 'Queens', 'Richmond', 'Bronx'])]

In [31]:
df.drop(['zip_code', 'address', 'city', 'neighborhood'], axis = 'columns', inplace = True)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8371 entries, 0 to 17140
Data columns (total 12 columns):
name                  8371 non-null object
beer_url              8371 non-null object
beer_type             8338 non-null object
abv                   8321 non-null float64
price                 8104 non-null float64
bar_url               8371 non-null object
origin                8207 non-null object
size                  8052 non-null float64
kind                  8350 non-null object
establishment_type    7931 non-null object
bar_name              8371 non-null object
county_name           8371 non-null object
dtypes: float64(3), object(9)
memory usage: 850.2+ KB


## Classify the Types of Beer

In [33]:
beer_categories = pd.read_csv('beer_types.csv')

In [34]:
df['beer_type'] = df.beer_type.str.strip()

In [36]:
df.loc[df.beer_type.isna(), 'beer_type'] = np.nan

In [37]:
df = df.merge(beer_categories, on = 'beer_type', how = 'left', validate = 'm:1')

In [39]:
df = df[df.beer_category != 'Other']

In [40]:
df.beer_category.value_counts(dropna = False)

IPA        1606
Ale        1232
Lager      1188
Stout       853
Fruit       820
Sour        716
Belgian     532
Wheat       428
Pilsner     407
Aged        126
NaN          61
Name: beer_category, dtype: int64

In [41]:
df.drop('beer_type', axis = 'columns', inplace = True)

## Origin

In [81]:
location_table = pd.read_excel('locations.xlsx')

In [82]:
location_table

,origin,origin_zone
0,"Brooklyn, NY",city
1,"St. Louis, MO",country
2,"New York, NY",city
3,"Portland, ME",country
4,NaN,NaN
...,...,...
483,"Kingston, , Jamaica,",world
484,"Jever, Lower Saxony",world
485,"Marshall, MI",country
486,"Millers Falls, MA",country


In [88]:
df = df.merge(location_table, on = 'origin')

In [95]:
df.drop('origin', axis = 'columns', inplace = True)

In [97]:
df.to_sql('beers_clean', db, index = False, )